In [1]:
from sagemaker import get_execution_role
import sagemaker
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from sagemaker import KMeans

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [21]:
train_input = TrainingInput(s3_data='s3://gcimlteam11/k-means/train/train_k-means_data_new.csv', content_type='text/csv')

In [2]:
s3_data='s3://gcimlteam11/k-means/train/train_k-means_data_new.csv'
import pandas as pd
s3_data =pd.read_csv(s3_data)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [3]:
# SageMaker 세션 및 역할 설정
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# S3 버킷 및 접두사 정의
bucket = 'gcimlteam11'
prefix = 'k-means'

# 학습 데이터 경로 설정
#train_input = TrainingInput(s3_data='s3://gcimlteam11/k-means/train/train_k-means_data_new.csv', content_type='text/csv')
train_input=s3_data
data_train = train_input 
data_train = data_train.astype('float32') 
num_clusters = 5 
output_location = 's3://' + bucket + '/model-artifacts'
kmeans = KMeans(role=role, 
               train_instance_count=1, 
               train_instance_type='ml.c4.xlarge', 
               output_path=f's3://{bucket}/{prefix}/output', 
               k=num_clusters)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
kmeans.fit(kmeans.record_set(data_train.values))

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: kmeans-2023-12-05-10-47-06-817


2023-12-05 10:47:07 Starting - Starting the training job...
2023-12-05 10:47:22 Starting - Preparing the instances for training.........
2023-12-05 10:49:05 Downloading - Downloading input data
2023-12-05 10:49:05 Training - Downloading the training image.........
2023-12-05 10:50:26 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[12/05/2023 10:50:32 INFO 139678672475968] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd"]', 'force_dense': 'true', '_disable_wait_to_read': 'false', '_enable_profiler': 'false', '_kvstore': 'auto'

In [5]:
kmeans_predictor = kmeans.deploy(initial_instance_count=1, 
                                 instance_type='ml.c4.xlarge')

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: kmeans-2023-12-05-10-51-39-756
INFO:sagemaker:Creating endpoint-config with name kmeans-2023-12-05-10-51-39-756
INFO:sagemaker:Creating endpoint with name kmeans-2023-12-05-10-51-39-756


-------!

In [6]:
result = kmeans_predictor.predict(data_train.values[0:len(data_train)])

cluster_labels = [r.label['closest_cluster'].float32_tensor.values[0] for r in result]

In [7]:
import pandas as pd
from sagemaker import KMeansPredictor
from sklearn.metrics import silhouette_score

test_data = pd.read_csv('s3://gcimlteam11/k-means/test/test_k-means_data_new.csv', header=None)

# 평가 데이터를 float32 형식으로 변환
data_eval = test_data.astype('float32')

# 클러스터 할당 받기
kmeans_predictor = KMeansPredictor(endpoint_name=kmeans_predictor.endpoint)
result = kmeans_predictor.predict(data_eval.values)

# 클러스터 레이블 추출
cluster_labels = [r.label['closest_cluster'].float32_tensor.values[0] for r in result]

# 실루엣 점수 계산
score = silhouette_score(data_eval, cluster_labels)

print(f"Silhouette Score: {score}")

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Silhouette Score: 0.42522937059402466


In [8]:
sagemaker.Session().delete_endpoint(kmeans_predictor.endpoint)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: kmeans-2023-12-05-10-51-39-756
